In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame(
    dict(
        user_id=np.random.randint(0, 1000, size=100000),
        n_imp=np.random.randint(1, 101, size=100000),
    )
)
df["n_click"] = df.n_imp.apply(lambda x: np.random.randint(0, int(np.ceil((x+1)*.3))))
df

,user_id,n_imp,n_click
0,962,41,7
1,132,75,16
2,750,58,0
3,449,28,2
4,731,9,2
...,...,...,...
99995,143,96,4
99996,279,34,9
99997,911,44,0
99998,305,32,5


In [3]:
from t_ab.aa import AATest
from t_ab.ctr import ImpressionBasedCTRTtest, UserBasedCTRTtest

ibtest = ImpressionBasedCTRTtest("user_id", "n_imp", "n_click")
ubtest = UserBasedCTRTtest("user_id", "n_imp", "n_click")
aa_test = AATest(
    {
        "Impression-based CTR": ibtest,
        "User-based CTR": ubtest,
    }
)

In [4]:
from typing import Generator
from sklearn.model_selection import train_test_split


def split_data(values: np.ndarray, n: int, random_state: int = 42) -> Generator[np.ndarray, None, None]:
    for i in range(n-1):
        vs, values = train_test_split(values, train_size=1/(n-i), random_state=random_state)
        yield vs
    yield values


def dfs_loader(df: pd.DataFrame, col: str, n_groups: int, n_tests: int=1000) -> Generator[list[pd.DataFrame], None, None]:
    values = df[col].unique()
    for i in range(n_tests):
        yield [df[df[col].isin(vs)] for vs in split_data(values, n_groups, i)]

In [5]:
aa_test_results = aa_test(dfs_loader(df, "user_id", 3, 1000))

In [6]:
for test_name, aa_test_result in aa_test_results.items():
    print(test_name)
    print(aa_test_result.multipletests_result, aa_test_result.is_rejected)

Impression-based CTR
(array([False, False, False]), array([0.80030312, 0.80030312, 0.77705159]), 0.016952427508441503, 0.016666666666666666) False
User-based CTR
(array([False, False, False]), array([0.88447726, 0.88447726, 0.88447726]), 0.016952427508441503, 0.016666666666666666) False


In [7]:
from t_ab.ab import ABTest

ab_test = ABTest(
    {
        "Impression-based CTR": ibtest,
        "User-based CTR": ubtest,
    }
)

In [8]:
for dfs in dfs_loader(df, "user_id", 3):
    ab_test_results = ab_test(dfs)
    break

In [9]:
from pprint import pprint

for test_name, ab_test_result in ab_test_results.items():
    print(test_name)
    pprint(ab_test_result)

Impression-based CTR
{'(0, 1)': CTRTestResult(statistics=(Statistics(mean=0.14748729184355866, std=0.010323717028923343, nobs=333), Statistics(mean=0.14767316921754495, std=0.010839354211857178, nobs=333)), ttest_result=TtestResult(statistic=-0.22659800271740016, pvalue=0.8208060850563259, is_rejected=False)),
 '(0, 2)': CTRTestResult(statistics=(Statistics(mean=0.14748729184355866, std=0.010323717028923343, nobs=333), Statistics(mean=0.14837787679410883, std=0.0103962025530665, nobs=334)), ttest_result=TtestResult(statistic=-1.1100557050153523, pvalue=0.2673762536353556, is_rejected=False)),
 '(1, 2)': CTRTestResult(statistics=(Statistics(mean=0.14767316921754495, std=0.010839354211857178, nobs=333), Statistics(mean=0.14837787679410883, std=0.0103962025530665, nobs=334)), ttest_result=TtestResult(statistic=-0.8568933964416413, pvalue=0.39181248494581045, is_rejected=False))}
User-based CTR
{'(0, 1)': CTRTestResult(statistics=(Statistics(mean=0.14751669548433902, std=0.0104743289365791